<h2>Protein pharmacophore selection using hydration-site analysis</h2>
<p>Gaokeng Xiao 2021-09-02</p>
change to working directory

In [1]:
import os
os.chdir('/public/gkxiao/work/ls4_gist')

create some helper to make things easy

In [2]:
from lxml import etree

# check pml file which includes a pharmacophore tag
def hasPh4(xml):
    root = xml.getroot()
    section = []
    for i in range(len(root)):
        section.append(root.getchildren()[i].tag)
    if 'pharmacophore' in section:
        return True
    else:
        return False

# gextract pharmacophore from pml file
def getPh4(xml):
    root = xml.getroot()
    section = []
    for i in range(len(root)):
        section.append(root.getchildren()[i].tag)
    idx = section.index('pharmacophore')
    ph4 = root.getchildren()[idx]
    return ph4

# return a feature list
def getFeature(ph4):
    features = []
    for iter in ph4:
        features.append(iter.tag)
    return features

to read a ligandscout pharmacophore model

In [3]:
pml = etree.parse('1ke8.pml')

In [4]:
# Dose this pml file includes a ph4 model? 
if hasPh4(pml) is False:
    print('This pml file does not include a pharmacophore model')
    exit
else:
    ph4 = getPh4(pml)

In [5]:
# list the types of feature
features = getFeature(ph4)
print(features)

['point', 'point', 'point', 'vector', 'vector', 'vector', 'vector', 'vector', 'vector', 'volume', 'volume', 'volume', 'volume', 'volume', 'volume', 'volume', 'volume', 'volume', 'volume', 'volume', 'volume', 'volume', 'volume', 'volume', 'volume', 'volume', 'volume', 'volume']


read the water denisity calculated with 3D-RISM

In [6]:
from gridData import Grid
g = Grid('1ke8_3drism_density.dx')

Protein pharmacophore selection using hydration-site analysis

In [7]:
for i in range(len(ph4)):
    # create feature
    feature_list = getFeature(ph4)
    f_type = feature_list[i]
    
    # Read feature
    f = ph4[i]
    # Read name name
    feat_name = f.items()[0][1]
    
    if f_type == 'point' or f_type == 'plane':
        # x y z coordination
        x=f[0].items()[0][1]
        y=f[0].items()[1][1]
        z=f[0].items()[2][1]
        
    elif f_type == 'vector' and feat_name == 'HBA':
        feat_name = 'HBA_target'
        f_origin = f[0]
        f_target = f[1]

        #x y z coordination
        x = f_target.items()[0][1]
        y = f_target.items()[1][1]
        z = f_target.items()[2][1]
    elif f_type == 'vector' and feat_name == 'HBD':
        feat_name = 'HBD_origin'
        f_origin = f[0]
        f_target = f[1]
        x = f_origin.items()[0][1]
        y = f_origin.items()[1][1]
        z = f_origin.items()[2][1]
    elif f_type == 'volume':
        x = f[0].items()[0][1]
        y = f[0].items()[1][1]
        z = f[0].items()[2][1]
    else:
        pass
    if f_type != 'volume':
        a=[]
        b=[]
        c=[]
        a.append(float(x))
        b.append(float(y))
        c.append(float(z))
        value = g.interpolated(a,b,c)
        print(feat_name,x,y,z,value[0])

H -10.37655 47.536613 9.012116 0.3586991705409551
H -7.421667 51.1267 15.606299 0.04042515679437711
H -13.282467 42.787968 9.254216 0.10889361245169017
HBA_target -10.819 52.155 11.823 9.889706906006275
HBA_target -9.244 45.091 6.068 3.234434937898573
HBA_target -11.648 45.394 5.712 3.075492435079222
HBA_target -11.648 45.394 5.712 3.075492435079222
HBD_origin -10.311 49.457 10.882 2.2363029801960863
HBD_origin -9.501 52.781 13.629 6.264434521935546
